In [1]:
'''
Write out information that is useful for setting up AMR-Wind sampling planes for FAST.Farm,
    following guidance from https://openfast.readthedocs.io/en/main/source/user/fast.farm/ModelGuidance.html
'''

'\nWrite out information that is useful for setting up AMR-Wind sampling planes for FAST.Farm,\n    following guidance from https://openfast.readthedocs.io/en/main/source/user/fast.farm/ModelGuidance.html\n'

In [2]:
import numpy as np

# User input

In [3]:
# ----------- Wind farm
wts  = {
          0 :{'x':1280.0,     'y':2560,       'z':0.0,  'D':126.9,  'zhub':86.5, 'cmax':5, 'fmax':10/6, 'Cmeander':1.9, 'name':'T0'},
          1 :{'x':1280.0,     'y':3200,       'z':0.0,  'D':126.9,  'zhub':86.5, 'cmax':5, 'fmax':10/6, 'Cmeander':1.9, 'name':'T1'},
          2 :{'x':1280.0,     'y':3840,       'z':0.0,  'D':126.9,  'zhub':86.5, 'cmax':5, 'fmax':10/6, 'Cmeander':1.9, 'name':'T2'},
        }

# ----------- Desired sweeps
# inflow_deg = [-5, 0, 5]  # inflow angle sweep. Determines how large the low-res box should be; TODO: Use this input

# ----------- Turbine parameters
# Set the yaw of each turbine for wind dir. One row for each wind direction.
# yaw_init = [ [0,0,0,0,0,0,0,0,0,0,0,0] ]  # TODO: Use this input

# ----------- AMR-Wind parameters
amr_fixed_dt = 0.25
amr_prob_lo = (0.0, 0.0, 0.0)
amr_prob_hi = (2560.0, 6400.0, 1280.0)
assert (amr_prob_hi[0] > amr_prob_lo[0]) & (amr_prob_hi[1] > amr_prob_lo[1]) & (amr_prob_hi[2] > amr_prob_lo[2]), "AMR-Wind high/low bounds misspecified!"
amr_n_cell = (256, 640, 128)
amr_max_level = 1  # Number of grid refinement levels

incflo_velocity_hh = (0.0, 10.0, 0.0)  # Hub-height velocity
postproc_name = 'sampling'


In [4]:
### Calculate AMR-Wind parameters that will be useful throughout
## AMR-Wind resolution
amr_dx0 = (amr_prob_hi[0] - amr_prob_lo[0]) / amr_n_cell[0]  # dx for Level 0
amr_dy0 = (amr_prob_hi[0] - amr_prob_lo[0]) / amr_n_cell[0]
amr_dz0 = (amr_prob_hi[0] - amr_prob_lo[0]) / amr_n_cell[0]
amr_ds0_max = max(amr_dx0, amr_dy0, amr_dz0)

amr_dx_refine = amr_dx0/(2**amr_max_level)  # dx for the maximum refinement Level
amr_dy_refine = amr_dy0/(2**amr_max_level)
amr_dz_refine = amr_dz0/(2**amr_max_level)
amr_ds_refine_max = max(amr_dx_refine, amr_dy_refine, amr_dz_refine)

In [5]:
### Calculate timestep output values and sampling frequency
## Low resolution domain, dt_lr
cmeander_min = float("inf")
Dwake_min = float("inf")
for turbkey in wts:
    cmeander_min = min(cmeander_min, wts[turbkey]['Cmeander'])
    Dwake_min = min(Dwake_min, wts[turbkey]['D'])  # Approximate D_wake as D_rotor
Vhub = np.sqrt(incflo_velocity_hh[0]**2 + incflo_velocity_hh[1]**2)

dt_lr_max = cmeander_min * Dwake_min / (10 * Vhub)
dt_lr = amr_fixed_dt * np.floor(dt_lr_max/amr_fixed_dt)  # Ensure that dt_lr is a multiple of the AMR-Wind timestep
assert dt_lr >= amr_fixed_dt, "AMR-Wind timestep too coarse for low resolution domain!"

## High resolution domain, dt_hr
fmax_max = 0
for turbkey in wts:
    fmax_max = max(0, wts[turbkey]['fmax'])
dt_hr_max = 1 / (2 * fmax_max)
dt_hr = amr_fixed_dt * np.floor(dt_hr_max/amr_fixed_dt)  # Ensure that dt_hr is a multiple of the AMR-Wind timestep
assert dt_hr >= amr_fixed_dt, "AMR-Wind timestep too coarse for high resolution domain!"
assert dt_hr <= dt_lr, "Low resolution timestep is finer than high resolution timestep!"

## Sampling frequency
out_output_frequency = int(dt_hr/amr_fixed_dt)

In [6]:
### Calculate grid resolutions
## Low resolution domain, ds_lr (s = x/y/z)
##   ASSUME: FAST.Farm LR zone uses Level 0 AMR-Wind grid spacing
##   NOTE: ds_lr is calculated independent of any x/y/z requirements,
##           just time step and velocity requiements
ds_lr_max = dt_lr * Vhub**2 / 15
ds_lr = amr_ds0_max * np.floor(ds_lr_max/amr_ds0_max)  # Ensure that ds_lr is a multiple of the coarse AMR-Wind grid spacing
assert ds_lr >= amr_dx0, "AMR-Wind Level 0 x-grid spacing too coarse for low resolution domain!"
assert ds_lr >= amr_dy0, "AMR-Wind Level 0 y-grid spacing too coarse for low resolution domain!"
assert ds_lr >= amr_dz0, "AMR-Wind Level 0 z-grid spacing too coarse for low resolution domain!"

## High resolution domain, ds_hr
##   ASSUME: FAST.Farm HR zone lies within the region of maxmum AMR-Wind grid refinement
##   NOTE: ds_hr is calculated independent of any x/y/z requirements,
##           just blade chord length requirements
cmax_min = float("inf")
for turbkey in wts:
    cmax_min = min(cmax_min, wts[turbkey]['cmax'])
ds_hr_max = cmax_min
ds_hr = amr_ds_refine_max * np.floor(ds_hr_max/amr_ds_refine_max)  # Ensure that ds_hr is a multiple of the refined AMR-Wind grid spacing
assert ds_hr >= amr_ds_refine_max, "AMR-Wind grid spacing too coarse for high resolution domain!"

In [7]:
### Calculate high-level info for sampling
out_labels = ["Low"]
for turbkey in wts:
    out_labels.append(f"High{wts[turbkey]['name']}_inflow0deg")
out_labels_str = " ".join(str(item) for item in out_labels)

print(f"# Sampling info generated by AMRWindSamplingCreation.py")
print(f"incflo.post_processing = {postproc_name} # averaging")
print(f"{postproc_name}.output_frequency = {out_output_frequency}")
print(f"{postproc_name}.fields = velocity # temperature tke")

print(f"{postproc_name}.labels = {out_labels_str}")

# Sampling info generated by AMRWindSamplingCreation.py
incflo.post_processing = sampling # averaging
sampling.output_frequency = 1
sampling.fields = velocity # temperature tke
sampling.labels = Low HighT0_inflow0deg HighT1_inflow0deg HighT2_inflow0deg


In [8]:
### Calculate low resolution grid placement
# Calculate minimum/maximum LR domain extents
wt_all_x_min = float("inf")     # Minimum x-value of any turbine
wt_all_x_max = -1*float("inf")
wt_all_y_min = float("inf")
wt_all_y_max = -1*float("inf")
wt_all_z_max = -1*float("inf")  # Tallest rotor disk point of any turbine
Drot_max = -1*float("inf")
for turbkey in wts:
    wt_all_x_min = min(wt_all_x_min, wts[turbkey]['x'])
    wt_all_x_max = max(wt_all_x_max, wts[turbkey]['x'])
    wt_all_y_min = min(wt_all_y_min, wts[turbkey]['y'])
    wt_all_y_max = max(wt_all_x_min, wts[turbkey]['y'])
    wt_all_z_max = max(wt_all_z_max, wts[turbkey]['zhub'] + 0.5*wts[turbkey]['D'])
    Drot_max = max(Drot_max, wts[turbkey]['D'])
    
x_buffer_lr = 3 * Drot_max
y_buffer_lr = 3 * Drot_max
z_buffer_lr = 1 * Drot_max
xlow_lr_min = wt_all_x_min - x_buffer_lr
xhigh_lr_max = wt_all_x_max + x_buffer_lr
ylow_lr_min = wt_all_y_min - y_buffer_lr
yhigh_lr_max = wt_all_y_max + y_buffer_lr
zhigh_lr_max = wt_all_z_max + z_buffer_lr

# Calculate the minimum/maximum LR domain coordinate lengths & number of grid cells
xdist_lr_min = xhigh_lr_max - xlow_lr_min  # Minumum possible length of x-extent of LR domain
xdist_lr = ds_lr * np.ceil(xdist_lr_min/ds_lr)  # The `+ ds_lr` comes from the +1 to NS_LOW in Sec. 4.2.15.6.4.1.1
# TODO: adjust xdist_lr calculation by also using `inflow_deg`
nx_lr = int(xdist_lr/ds_lr) + 1

ydist_lr_min = yhigh_lr_max - ylow_lr_min
ydist_lr = ds_lr * np.ceil(ydist_lr_min/ds_lr)
# TODO: adjust ydist_lr calculation by also using `inflow_deg`
ny_lr = int(ydist_lr/ds_lr) + 1

zdist_lr = ds_lr * np.ceil(zhigh_lr_max/ds_lr)
nz_lr = int(zdist_lr/ds_lr) + 1

# Calculate actual LR domain extent
#  NOTE: Sampling planes should measure at AMR-Wind cell centers, not cell edges
#  NOTE: Should I be using dx/dy/dz values here or ds_lr?
#          - I think it's correct to use ds_lr to get to the xlow values,
#              but then offset by 0.5*amr_dx0
xlow_lr = ds_lr * np.floor(xlow_lr_min/ds_lr) - 0.5*amr_dx0
xhigh_lr = xlow_lr + xdist_lr
ylow_lr = ds_lr * np.floor(ylow_lr_min/ds_lr) - 0.5*amr_dy0
yhigh_lr = ylow_lr + ydist_lr
zlow_lr = 0.5 * amr_dz0  # Lowest z point is half the height of the lowest grid cell
zhigh_lr = zlow_lr + zdist_lr
zoffsets_lr = np.arange(zlow_lr, zhigh_lr+ds_lr, ds_lr) - zlow_lr
zoffsets_lr_str = " ".join(str(int(item)) for item in zoffsets_lr)

assert xhigh_lr < amr_prob_hi[0], "LR domain extends beyond maximum AMR-Wind x-extent!"
assert xlow_lr > amr_prob_lo[0], "LR domain extends beyond minimum AMR-Wind x-extent!"
assert yhigh_lr < amr_prob_hi[1], "LR domain extends beyond maximum AMR-Wind y-extent!"
assert ylow_lr > amr_prob_lo[1], "LR domain extends beyond minimum AMR-Wind y-extent!"
assert zhigh_lr < amr_prob_hi[2], "LR domain extends beyond maximum AMR-Wind z-extent!"
assert zlow_lr > amr_prob_lo[2], "LR domain extends beyond minimum AMR-Wind z-extent!"

# Reformat info for AMR-Wind input file
print(f"# Low sampling grid spacing: {ds_lr} m")
print(f"{postproc_name}.Low.type         = PlaneSampler")
print(f"{postproc_name}.Low.num_points   = {nx_lr} {ny_lr}")
print(f"{postproc_name}.Low.origin       = {xlow_lr:.1f} {ylow_lr:.1f}")  # Round the float output
print(f"{postproc_name}.Low.axis1        = {xdist_lr:.1f} 0.0 0.0")  # Assume: axis1 oriented parallel to AMR-Wind x-axis
print(f"{postproc_name}.Low.axis2        = 0.0 {ydist_lr:.1f} 0.0")  # Assume: axis2 oriented parallel to AMR-Wind y-axis
print(f"{postproc_name}.Low.normal       = 0.0 0.0 1.0")
print(f"{postproc_name}.Low.offsets      = {zoffsets_lr_str}")

# Low sampling grid spacing: 10.0 m
sampling.Low.type         = PlaneSampler
sampling.Low.num_points   = 78 206
sampling.Low.origin       = 885.0 2165.0
sampling.Low.axis1        = 770.0 0.0 0.0
sampling.Low.axis2        = 0.0 2050.0 0.0
sampling.Low.normal       = 0.0 0.0 1.0
sampling.Low.offsets      = 0 10 20 30 40 50 60 70 80 90 100 110 120 130 140 150 160 170 180 190 200 210 220 230 240 250 260 270 280


In [9]:
### Calculate high resolution grid placements
turbkey = 0
wt_x = wts[turbkey]['x']
wt_y = wts[turbkey]['y']
wt_z = wts[turbkey]['zhub'] + 0.5*wts[turbkey]['D']
wt_D = wts[turbkey]['D']
wt_name = wts[turbkey]['name']

# Calculate minimum/maximum HR domain extents
x_buffer_hr = 0.6 * wt_D
y_buffer_hr = 0.6 * wt_D
z_buffer_hr = 0.6 * wt_D

xlow_hr_min = wt_x - x_buffer_hr
xhigh_hr_max = wt_x + x_buffer_hr
ylow_hr_min = wt_y - y_buffer_hr
yhigh_hr_max = wt_y + y_buffer_hr
zhigh_hr_max = wt_z + z_buffer_hr

# Calculate the minimum/maximum HR domain coordinate lengths & number of grid cells
xdist_hr_min = xhigh_hr_max - xlow_hr_min  # Minumum possible length of x-extent of HR domain
xdist_hr = ds_hr * np.ceil(xdist_hr_min/ds_hr)  
nx_hr = int(xdist_hr/ds_hr) + 1

ydist_hr_min = yhigh_hr_max - ylow_hr_min
ydist_hr = ds_hr * np.ceil(ydist_hr_min/ds_hr)
ny_hr = int(ydist_hr/ds_hr) + 1

zdist_hr = ds_hr * np.ceil(zhigh_hr_max/ds_hr)
nz_hr = int(zdist_hr/ds_hr) + 1

# Calculate actual HR domain extent
#  NOTE: Sampling planes should measure at AMR-Wind cell centers, not cell edges
xlow_hr = ds_hr * np.floor(xlow_hr_min/ds_hr) - 0.5*amr_dx_refine
xhigh_hr = xlow_hr + xdist_hr
ylow_hr = ds_hr * np.floor(ylow_hr_min/ds_hr) - 0.5*amr_dy_refine
yhigh_hr = ylow_hr + ydist_hr
zlow_hr = zlow_lr
zhigh_hr = zlow_hr + zdist_hr
zoffsets_hr = np.arange(zlow_hr, zhigh_hr+ds_hr, ds_hr) - zlow_hr
zoffsets_hr_str = " ".join(str(int(item)) for item in zoffsets_hr)

assert xhigh_hr < amr_prob_hi[0], f"HR domain for {wt_name} extends beyond maximum AMR-Wind x-extent!"
assert xlow_hr > amr_prob_lo[0], f"HR domain for {wt_name} extends beyond minimum AMR-Wind x-extent!"
assert yhigh_hr < amr_prob_hi[1], f"HR domain for {wt_name} extends beyond maximum AMR-Wind y-extent!"
assert ylow_hr > amr_prob_lo[1], f"HR domain for {wt_name} extends beyond minimum AMR-Wind y-extent!"
assert zhigh_hr < amr_prob_hi[2], f"HR domain for {wt_name} extends beyond maximum AMR-Wind z-extent!"
assert zlow_hr > amr_prob_lo[2], f"HR domain for {wt_name} extends beyond minimum AMR-Wind z-extent!"

# Reformat info for AMR-Wind input file
print(f"# Turbine {wt_name} at (x,y) = ({wt_x}, {wt_y}), with D = {wt_D}, grid spacing = {ds_hr}")
print(f"{postproc_name}.{wt_name}.type         = PlaneSampler")
print(f"{postproc_name}.{wt_name}.num_points   = {nx_hr} {ny_hr}")
print(f"{postproc_name}.{wt_name}.origin       = {xlow_hr:.1f} {ylow_hr:.1f}")  # Round the float output
print(f"{postproc_name}.{wt_name}.axis1        = {xdist_hr:.1f} 0.0 0.0")  # Assume: axis1 oriented parallel to AMR-Wind x-axis
print(f"{postproc_name}.{wt_name}.axis2        = 0.0 {ydist_hr:.1f} 0.0")  # Assume: axis2 oriented parallel to AMR-Wind y-axis
print(f"{postproc_name}.{wt_name}.normal       = 0.0 0.0 1.0")
print(f"{postproc_name}.{wt_name}.offsets      = {zoffsets_hr_str}")

# Turbine T0 at (x,y) = (1280.0, 2560), with D = 126.9, grid spacing = 5.0
sampling.T0.type         = PlaneSampler
sampling.T0.num_points   = 32 32
sampling.T0.origin       = 1197.5 2477.5
sampling.T0.axis1        = 155.0 0.0 0.0
sampling.T0.axis2        = 0.0 155.0 0.0
sampling.T0.normal       = 0.0 0.0 1.0
sampling.T0.offsets      = 0 5 10 15 20 25 30 35 40 45 50 55 60 65 70 75 80 85 90 95 100 105 110 115 120 125 130 135 140 145 150 155 160 165 170 175 180 185 190 195 200 205 210 215 220 225 230
